<a href="https://colab.research.google.com/github/cbikkanur/twitter/blob/master/documentSimilarity_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [100]:
import numpy as np
import pandas as pd
import csv
import os
import time
import datetime
print(os.listdir("."))
from scipy.spatial import distance

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.pipeline import Pipeline
np.random.seed(0)

import warnings
warnings.filterwarnings('ignore')


['.config', 'drive', 'test_vectors.npy', 'BERT_similar_test_tweets.csv', 'similar_test_tweets.csv', 'BERT_nearest_test_tweets.csv', 'train_vectors.npy', 'sample_data']


In [101]:
import tensorflow as tf
# Go to Edit > Notebook Settings > Hardware accelerator > GPU
device_name = tf.test.gpu_device_name()
print(device_name)

import torch
if torch.cuda.is_available():
  device = torch.device('cuda')
  print('We will use {} {}GPU(s)'.format(torch.cuda.device_count(), torch.cuda.get_device_name() ))
else:
  device = torch.device('cuda')
  print('No GPU(s) available; we will use "cpu"')

device=torch.device('cuda')

/device:GPU:0
We will use 1 Tesla P100-PCIE-16GBGPU(s)


In [37]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
DRIVE_PATH = "/content/drive/My Drive/MIDS/docSimilarity/"

print(os.listdir(DRIVE_PATH))

['CosineSimilarity.ipynb', 'getTwitterData_TweePy.ipynb', 'cbikkanur_twitter_credentials.json', 'cbikkanur_twitter_credentials.ipynb', 'tweets_04232020.csv', 'SearchTweets.csv', 'BankTweets.csv', 'documentSimilarity.ipynb', 'tweets_04242020.csv', 'SearchTweets_04242020.csv', 'BankTweets_04242020.csv', 'SearchTweets_04252020.csv', 'tweets_04252020.csv', 'BankTweets_04252020.csv', 'documentSimilarity_TfIdf.ipynb.txt', 'SearchTweets_04262020.csv', 'BankTweets_04262020.csv', 'tweets_04262020.csv', '.ipynb_checkpoints', 'model_save', 'train_vectors.npy', 'test_vectors.npy', 'getTwitterData.ipynb', 'SearchTweets_04292020.csv', 'BankTweets_04292020.csv', 'tweets_04292020.csv', 'TfIdf_similar_test_tweets.csv', 'documentSimilarity_TfIdf.ipynb']


In [102]:
all_files = [DRIVE_PATH+'tweets_04232020.csv', DRIVE_PATH+'./tweets_04242020.csv', DRIVE_PATH+'./tweets_04252020.csv', DRIVE_PATH+'./tweets_04262020.csv', DRIVE_PATH+'./tweets_04292020.csv']

df_list = []
for filename in all_files:
    df_list.append(pd.read_csv(filename, sep=',',header=0, encoding='utf-8', index_col = 0))
    
df_tweets = pd.concat(df_list, ignore_index=True)
df_tweets    

,Tweet_Id,User_Id,User_Name,User_Screen_Name,Theme,Text
0,1253375593271394312,80374332,Citibank,Citibank,Bank/Financial,Registering for online access and activating y...
1,1253322747851296768,80374332,Citibank,Citibank,Bank/Financial,Protect your CARES Act payments: Validate comm...
2,1253066045700681731,80374332,Citibank,Citibank,Bank/Financial,It’s simple to set up a payment account in the...
3,1252960358454767616,80374332,Citibank,Citibank,Bank/Financial,We’re committed to helping provide the support...
4,1252673469852143618,80374332,Citibank,Citibank,Bank/Financial,Protect Yourself from COVID-19 Scams: Don’t re...
...,...,...,...,...,...,...
14995,1255520212260175872,1051372512582328321,vinlywrongar,vinlywrongar,coffee,RT @lucxs__x: You're the coffee that I need in...
14996,1255520211610075136,16383553,janmaxwell,janmaxwell,coffee,RT @RoberteLove: I had to go to the store this...
14997,1255520210041348096,547284373,Beth Ann Cook,revbethanncook,coffee,@mykalmphoto Nice photos. When this is all ov...
14998,1255520209500323842,355634795,first name crap second name bag,blaairee,coffee,RT @minturnalexandr: Ten years ago I bought th...


In [103]:
df_tweets.drop_duplicates(["Text", "User_Id"], inplace=True) # remove duplicates in place and reset index
df_tweets = df_tweets.reset_index(drop=True)
df_tweets.shape

(7958, 6)

In [104]:
df = pd.DataFrame()
df['text'] = df_tweets['Text']
df.tail()

,text
7953,RT @lucxs__x: You're the coffee that I need in...
7954,RT @RoberteLove: I had to go to the store this...
7955,@mykalmphoto Nice photos. When this is all ov...
7956,RT @minturnalexandr: Ten years ago I bought th...
7957,RT @IvankaTrump: Small businesses like Amy Wri...


In [105]:
total_indices_size = df.shape[0]
train_size = 0.8
train_indices_size = int(train_size * total_indices_size)
test_indices_size = total_indices_size - train_indices_size
print('total records: {}\n trainig_set record: {}\n test_set records: {}'.format(total_indices_size, train_indices_size, test_indices_size))

total_indices_array = np.array([x for x in range(total_indices_size)])
np.random.shuffle(total_indices_array)
train_indices, test_indices = total_indices_array[:train_indices_size], total_indices_array[train_indices_size:]

total records: 7958
 trainig_set record: 6366
 test_set records: 1592


In [106]:
train_set, test_set = df.loc[train_indices], df.loc[test_indices]
train_set[:5]

,text
2459,@ImHerMajesty Get a truck. I want my next car ...
7208,(That's a cake using LEGO bricks. Not an actua...
3067,@J_C_Shin Hi thanks for reaching out. It's imp...
5900,RT @LetsWeeb: Why does this dog look like it j...
274,"@theliftsavage Hi. No, this is not something w..."


In [44]:
# Documentation for transformers: https://huggingface.co/transformers/index.html
! pip install transformers

In [45]:
import torch
from transformers import BertTokenizer, BertModel, BertForSequenceClassification
from keras.preprocessing.sequence import pad_sequences

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', output_hidden_states = True ) # return hidden layers
model.cuda

<bound method Module.cuda of BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNo

In [0]:
def vectorize_sentence(tokenizer, model, text):
  MAX_LENGTH = 140
  input_ids = tokenizer.encode(text, add_special_tokens=True, max_length=MAX_LENGTH) # encode for tokenizing
  padded_sequences = pad_sequences([input_ids], maxlen=MAX_LENGTH, dtype='long', truncating='post', padding='post') # pad 0 at the end of sentence until MAX_LENGTH
  input_ids = padded_sequences[0] # remove the outer list
  attention_mask = [int(i>0) for i in input_ids]

  input_ids = torch.tensor(input_ids)
  attention_mask = torch.tensor(attention_mask)

  input_ids = input_ids.unsqueeze(0) # Extra dimension for batch
  attention_mask = attention_mask.unsqueeze(0)  # Extra dimension for batch

  model.eval() # Only forward pass
  input_ids.to(device) # https://github.com/huggingface/transformers/blob/master/notebooks/02-transformers.ipynb
  attention_mask.to(device)

  with torch.no_grad(): # No gradient descent n
    logits, encoded_layers = model(input_ids = input_ids, token_type_ids = None, attention_mask = attention_mask)
    last_layer = 12
    batch = 0
    last_token_index = 0 

    vector = encoded_layers[last_layer][batch][last_token_index]
    vector = vector.detach().cpu().numpy()

    return vector

In [47]:
sample_text = test_set.iloc[2].text
sample_tokens = tokenizer.tokenize(sample_text)
print('sample text: \n{} \n\n BERT tokens for sample text: \n{}'.format(sample_text, sample_tokens))

sample text: 
@roth_jroth1515 Hi Jennifer. Do you have a moment to Live Chat? ^B 

 BERT tokens for sample text: 
['@', 'roth', '_', 'jr', '##oth', '##15', '##15', 'hi', 'jennifer', '.', 'do', 'you', 'have', 'a', 'moment', 'to', 'live', 'chat', '?', '^', 'b']


In [48]:
sample_vector = vectorize_sentence(tokenizer, model, sample_text)
print('Shape of the vector: {}'.format(sample_vector.shape))

Shape of the vector: (768,)


In [0]:
def format_time(elapsed):
  return str(datetime.timedelta(seconds = int(round(elapsed))))


In [0]:
def vectorize_dataset(df):
  start_time = time.time()
  embeddings = []
  num_records = df.shape[0]
  row_num = 0
  for index, row in df.iterrows():
    text = row['text']
    if row_num % 200 == 0 and not row_num == 0:
      elapsed = format_time(time.time() - start_time)
      rows_per_sec = (time.time() - start_time) / row_num
      remaining_sec = rows_per_sec * (num_records - row_num)
      remaining = format_time(remaining_sec)
      print(' comment {:>7,} of {:>7,}.    Elapsed: {:}. Remaining: {:}'.format(row_num, num_records, elapsed, remaining))
    embeddings.append(vectorize_sentence(tokenizer, model, text))
    row_num += 1
  return  embeddings 

In [51]:

train_set_embeddings = vectorize_dataset(train_set)
train_vectors = np.stack(train_set_embeddings)
train_vectors.shape

 comment     200 of   6,366.    Elapsed: 0:01:35. Remaining: 0:48:39
 comment     400 of   6,366.    Elapsed: 0:03:09. Remaining: 0:47:03
 comment     600 of   6,366.    Elapsed: 0:04:44. Remaining: 0:45:24
 comment     800 of   6,366.    Elapsed: 0:06:17. Remaining: 0:43:45
 comment   1,000 of   6,366.    Elapsed: 0:07:49. Remaining: 0:41:55
 comment   1,200 of   6,366.    Elapsed: 0:09:19. Remaining: 0:40:08
 comment   1,400 of   6,366.    Elapsed: 0:10:50. Remaining: 0:38:27
 comment   1,600 of   6,366.    Elapsed: 0:12:20. Remaining: 0:36:44
 comment   1,800 of   6,366.    Elapsed: 0:13:51. Remaining: 0:35:08
 comment   2,000 of   6,366.    Elapsed: 0:15:23. Remaining: 0:33:34
 comment   2,200 of   6,366.    Elapsed: 0:16:56. Remaining: 0:32:03
 comment   2,400 of   6,366.    Elapsed: 0:18:28. Remaining: 0:30:31
 comment   2,600 of   6,366.    Elapsed: 0:20:02. Remaining: 0:29:00
 comment   2,800 of   6,366.    Elapsed: 0:21:35. Remaining: 0:27:30
 comment   3,000 of   6,366.    El

(6366, 768)

In [52]:
test_set_embeddings = vectorize_dataset(test_set)
test_vectors = np.stack(test_set_embeddings)
test_vectors.shape


 comment     200 of   1,592.    Elapsed: 0:01:34. Remaining: 0:10:55
 comment     400 of   1,592.    Elapsed: 0:03:09. Remaining: 0:09:22
 comment     600 of   1,592.    Elapsed: 0:04:44. Remaining: 0:07:49
 comment     800 of   1,592.    Elapsed: 0:06:19. Remaining: 0:06:15
 comment   1,000 of   1,592.    Elapsed: 0:07:54. Remaining: 0:04:40
 comment   1,200 of   1,592.    Elapsed: 0:09:29. Remaining: 0:03:06
 comment   1,400 of   1,592.    Elapsed: 0:11:04. Remaining: 0:01:31


(1592, 768)

In [53]:
output_dir = './drive/My Drive/MIDS/docSimilarity/model_save/'
print(os.listdir("."))
if not os.path.exists(output_dir):
  print('directory does not exist')
  os.makedirs(output_dir)

np.save('train_vectors.npy', train_vectors)  
np.save('test_vectors.npy', test_vectors)  

! cp -r ./train_vectors.npy './drive/My Drive/MIDS/docSimilarity/model_save/'
! cp -r ./test_vectors.npy './drive/My Drive/MIDS/docSimilarity/model_save/'

['.config', 'drive', 'test_vectors.npy', 'similar_test_tweets.csv', 'BERT_nearest_test_tweets.csv', 'train_vectors.npy', 'sample_data']


# Cosine Similarity:

In [0]:
def print_similar_texts(query_text, query_text_vector,  train_matrix, train_indices, df_tweets, n = 5):
    cosine_similarities_n = cosine_similarity(query_text_vector, train_matrix).flatten()
    top_Indices = cosine_similarities_n.argsort()[::-1][:n]
    top_tweet_Indices = train_indices[top_Indices]
    
    print('\nInput Text:\n {} \n'.format(query_text))
    for index, sim_text in enumerate(df_tweets.loc[top_tweet_Indices, "Text"]):
        print('=' * 30, 'Similar Text: {} || Similar Score: {}'.format(index+1, np.round(cosine_similarities_n[top_Indices[index]], 3)), '=' * 30, '\n')
        print(sim_text, '\n')

In [81]:
query_index = 16
query_text = df_tweets.loc[test_indices[query_index], "Text"]
query_text_vector = test_vectors[query_index].reshape(1, -1)
print_similar_texts(query_text, query_text_vector, train_vectors, train_indices, df_tweets, n = 5)


Input Text:
 coffee tastes so good at night🖤 

============================== Similar Text: 1 || Similar Score: 0.9330000281333923 ============================== 

I’ve been making my own iced coffee most days and its sooo good 😋 

============================== Similar Text: 2 || Similar Score: 0.9120000004768372 ============================== 

my cake came out so fire 😭😭🤤🤤 

============================== Similar Text: 3 || Similar Score: 0.9100000262260437 ============================== 

I mean iced coffee sounds good too but the whipped cream from there 😋😋😋 

============================== Similar Text: 4 || Similar Score: 0.9089999794960022 ============================== 

Coffee ain’t hittin no more 😫 

============================== Similar Text: 5 || Similar Score: 0.9049999713897705 ============================== 

@Shithotish @Coffee_Ink_ Should be sat in my pair of acres😅 



In [0]:
def find_similar_n(train_matrix, train_indices, test_matrix, test_indices, test_set_df, df_tweets, n=5):
    similar_texts_list = []
    similar_scores_list = []
    similar_top_score_list = []
    similar_themes_list = []
    similar_top_themes_list = []
    for vector in test_matrix:
        cosine_similarities_n = cosine_similarity(vector.reshape(1, -1), train_matrix).flatten()
        similar_doc_indices =  cosine_similarities_n.argsort()[::-1][:n]
        top_tweet_Indices = train_indices[similar_doc_indices]
        
        similar_texts = [text for text in df_tweets.loc[top_tweet_Indices, "Text"]]
        similar_texts_list.append(similar_texts)
        
        similar_scores = [np.round(score, 3) for score in cosine_similarities_n[similar_doc_indices]]
        similar_scores_list.append(similar_scores)
        similar_top_score_list.append(similar_scores[0])
                                                                   
        similar_themes = [theme for theme in df_tweets.loc[top_tweet_Indices, "Theme"]]
        similar_themes_list.append(similar_themes)
        
        top_similar_theme = max(set(similar_themes), key=lambda x: similar_themes.count(x))
        similar_top_themes_list.append(top_similar_theme)
    
    df = test_set_df.copy() 
    df["original_theme"] = df_tweets.loc[test_indices, "Theme"]
    df["similar_texts"] = similar_texts_list
    df["similar_scores"] = similar_scores_list  
    df["top_similar_score"] = similar_top_score_list
    df["similar_themes"] = similar_themes_list
    df["top_similar_themes"] = similar_top_themes_list
    return df

In [107]:
similar_test_df = find_similar_n(train_vectors, train_indices, test_vectors, test_indices, test_set, df_tweets, n=5)
similar_test_df.head()

,text,original_theme,similar_texts,similar_scores,top_similar_score,similar_themes,top_similar_themes
6953,"RT @Essence: Join us TOMORROW, 4/27 at 7 PM ES...",Bank/Financial,[RT @HaggertyColleen: Amazing conversation w @...,"[0.955, 0.953, 0.952, 0.951, 0.944]",0.955,"[Bank/Financial, Bank/Financial, Bank/Financia...",Bank/Financial
6031,RT @Jordan_Sather_: I can (almost) guarantee y...,nasa,[RT @Jordan_Sather_: I can (almost) guarantee ...,"[1.0, 1.0, 1.0, 1.0, 1.0]",1.000,"[nasa, nasa, nasa, nasa, nasa]",nasa
239,@roth_jroth1515 Hi Jennifer. Do you have a mom...,Bank/Financial,[@PRDR123 Just following up with you. Do you s...,"[0.968, 0.967, 0.965, 0.965, 0.964]",0.968,"[Bank/Financial, Bank/Financial, Bank/Financia...",Bank/Financial
5720,RT @gguksluver: jimins face when he realized j...,cake,[RT @gguksluver: jimins face when he realized ...,"[1.0, 1.0, 0.956, 0.952, 0.952]",1.000,"[cake, cake, cake, cake, cake]",cake
7102,RT @SaralPatel: Shri @RahulGandhi will be inte...,covid-19,[RT @INCIndia: Shri @RahulGandhi will be inter...,"[0.996, 0.996, 0.988, 0.919, 0.917]",0.996,"[covid-19, covid-19, covid-19, covid-19, covid...",covid-19


In [108]:
accuracy = 100 * np.sum(similar_test_df["original_theme"] == similar_test_df["top_similar_themes"])/similar_test_df.shape[0]
print("Accuracy on test data in predicting theme: {}%".format(np.round(accuracy,2)))

Accuracy on test data in predicting theme: 72.55%


In [0]:
similar_test_df.to_csv("BERT_similar_test_tweets.csv")

In [0]:
os.listdir('.')
! cp -r ./BERT_similar_test_tweets.csv './drive/My Drive/MIDS/docSimilarity/model_save/'

# Nearest Euclidean:

In [0]:
def print_nearest_euclidean_texts(query_text, query_text_vector,  train_matrix, train_indices, df_tweets, n = 5):
    euclidean_distance = distance.cdist(sample_vector.reshape(1, -1), train_vectors, 'euclidean').flatten() 
    top_Indices = euclidean_distance.argsort()[:n]
    top_tweet_Indices = train_indices[top_Indices]
    
    print('\nInput Text:\n {} \n'.format(query_text))
    for index, sim_text in enumerate(df_tweets.loc[top_tweet_Indices, "Text"]):
        print('=' * 30, 'Similar Text: {} || Euclidean Distance: {}'.format(index+1, np.round(euclidean_distance[top_Indices[index]], 3)), '=' * 30, '\n')
        print(sim_text, '\n')

In [117]:
query_index = 16
query_text = df_tweets.loc[test_indices[query_index], "Text"]
query_text_vector = test_vectors[query_index].reshape(1, -1)
print_nearest_euclidean_texts(query_text, query_text_vector, train_vectors, train_indices, df_tweets, n = 5)


Input Text:
 coffee tastes so good at night🖤 

============================== Similar Text: 1 || Euclidean Distance: 3.701 ============================== 

@PRDR123 Just following up with you. Do you still need assistance? ^B 

============================== Similar Text: 2 || Euclidean Distance: 3.762 ============================== 

@PRDR123 Hi Peter. Thank you for the feedback. Is there something I can help you with? ^B 

============================== Similar Text: 3 || Euclidean Distance: 3.8 ============================== 

@PabloLlama88 Please try your call again at a later time. Thank you. ^B 

============================== Similar Text: 4 || Euclidean Distance: 3.886 ============================== 

@mcsean123 Hi there! Are you available to chat? ^Vee 

============================== Similar Text: 5 || Euclidean Distance: 3.89 ============================== 

@kgmerz Hi Karen. I would be happy to look into this. Do you have time to chat securely off Twitter so I can look at 

In [0]:
def find_nearest_n(train_matrix, train_indices, test_matrix, test_indices, test_set_df, df_tweets, n=5):
    similar_texts_list = []
    euclidean_distances_list = []
    euclidean_nearest_distance_list = []   
    similar_themes_list = []
    similar_top_themes_list = []
    for vector in test_matrix:
        euclidean_distance = distance.cdist(vector.reshape(1, -1), train_vectors, 'euclidean').flatten()
        nearest_doc_indices =  euclidean_distance.argsort()[:n]        
        top_tweet_Indices = train_indices[nearest_doc_indices]
        
        similar_texts = [text for text in df_tweets.loc[top_tweet_Indices, "Text"]]
        similar_texts_list.append(similar_texts)
        
        euclidean_distances = [np.round(score, 3) for score in euclidean_distance[nearest_doc_indices]]
        euclidean_distances_list.append(euclidean_distances)
        euclidean_nearest_distance_list.append(euclidean_distances[0])
                                                                   
        similar_themes = [theme for theme in df_tweets.loc[top_tweet_Indices, "Theme"]]
        similar_themes_list.append(similar_themes)
        
        top_similar_theme = max(set(similar_themes), key=lambda x: similar_themes.count(x))
        similar_top_themes_list.append(top_similar_theme)
    
    df = test_set_df.copy() 
    df["original_theme"] = df_tweets.loc[test_indices, "Theme"]
    df["similar_texts"] = similar_texts_list
    df["euclidean_distances"] = euclidean_distances_list  
    df["nearest_euclidean_distance"] = euclidean_nearest_distance_list
    df["similar_themes"] = similar_themes_list
    df["top_similar_themes"] = similar_top_themes_list
    return df

In [121]:
nearest_test_df = find_nearest_n(train_vectors, train_indices, test_vectors, test_indices, test_set, df_tweets)
nearest_test_df.head()

,text,original_theme,similar_texts,euclidean_distances,nearest_euclidean_distance,similar_themes,top_similar_themes
6953,"RT @Essence: Join us TOMORROW, 4/27 at 7 PM ES...",Bank/Financial,[RT @HaggertyColleen: Amazing conversation w @...,"[4.126, 4.255, 4.258, 4.293, 4.663]",4.126,"[Bank/Financial, Bank/Financial, Bank/Financia...",Bank/Financial
6031,RT @Jordan_Sather_: I can (almost) guarantee y...,nasa,[RT @Jordan_Sather_: I can (almost) guarantee ...,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.000,"[nasa, nasa, nasa, nasa, nasa]",nasa
239,@roth_jroth1515 Hi Jennifer. Do you have a mom...,Bank/Financial,[@PRDR123 Just following up with you. Do you s...,"[3.701, 3.762, 3.8, 3.886, 3.89]",3.701,"[Bank/Financial, Bank/Financial, Bank/Financia...",Bank/Financial
5720,RT @gguksluver: jimins face when he realized j...,cake,[RT @gguksluver: jimins face when he realized ...,"[0.0, 0.0, 4.094, 4.297, 4.297]",0.000,"[cake, cake, cake, dog, dog]",cake
7102,RT @SaralPatel: Shri @RahulGandhi will be inte...,covid-19,[RT @INCIndia: Shri @RahulGandhi will be inter...,"[1.338, 1.338, 2.242, 5.838, 5.891]",1.338,"[covid-19, covid-19, covid-19, covid-19, covid...",covid-19


In [122]:
accuracy = 100 * np.sum(nearest_test_df["original_theme"] == nearest_test_df["top_similar_themes"])/nearest_test_df.shape[0]
print("Accuracy on test data in predicting theme: {}%".format(np.round(accuracy,2)))

Accuracy on test data in predicting theme: 72.68%


In [0]:
nearest_test_df.to_csv("BERT_nearest_test_tweets.csv")
! cp -r ./BERT_nearest_test_tweets.csv './drive/My Drive/MIDS/docSimilarity/model_save/'